In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-05-25


Found rundate 2024-05-24 10:00:02 < 2024-05-25 at commit 74f3a2e5; returning commit 7149e8a


'2024-05-25 10:00:03'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-05-25 10:00:03, NJSP was reporting 241 YTD deaths
Current YTD Deaths (2025-05-25 10:00:00-04:00): 204
Previous year YTD Deaths (adjusted; 2024-05-25 10:00:03): 239.99789318338327
Projected 2025 total: 621.659799119738
-15.0% change


3ce83d25431cf11eb2c401179a26cc39ee06635b: FAUQStats cache miss: 2024, Sat May 25 10:00:03 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,STREET,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,1,State Highway 1,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,129,State Highway 129,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,446,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,NaN,East Oak St,1.0,1.0,0.0,0.0,0.0,East Oak St,NaN,2024-01-05 14:12:00-05:00
12927,20,Union,2009,Linden City,27,State Highway 27,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-06 08:37:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13188,08,Gloucester,0810,Mantua Twsp,635,County 635 MP 2.81,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-05-21 12:01:00-04:00
13189,11,Mercer,1106,Hopewell Twsp,518,County 518,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-05-21 15:02:00-04:00
13194,02,Bergen,0223,Hackensack City,80,Interstate 80 MP 66.6,2.0,1.0,1.0,0.0,0.0,NaN,NaN,2024-05-21 23:43:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,8,0,4,0,11
Bergen,6,1,2,0,9
Burlington,9,3,3,0,14
Camden,10,2,6,0,17
Cape May,2,0,0,0,2
Cumberland,3,2,2,0,6
Essex,5,1,8,1,15
Gloucester,7,4,2,0,10
Hudson,2,1,2,0,4


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

39.6% through the year, 60.4% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,9,31,8
Bergen,driver,8,14,6
Burlington,driver,14,31,9
Camden,driver,8,19,10
Cape May,driver,1,8,2
...,...,...,...,...
Salem,crashes,4,14,7
Somerset,crashes,4,18,6
Sussex,crashes,2,12,1


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,41,1,29,1,15
Bergen,26,1,11,2,10
Burlington,37,1,24,4,11
Camden,34,0,22,4,10
Cape May,13,0,12,0,1
Cumberland,23,0,14,9,9
Essex,47,2,17,6,28
Gloucester,33,0,20,12,7
Hudson,18,0,10,1,10


In [9]:
projected.to_csv(PROJECTED_CSV)